In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
historical_transaction_data = pd.read_csv("/kaggle/input/datastorm-competition/Historical-transaction-data.csv")
store_info_data = pd.read_csv("/kaggle/input/datastorm-competition/Store-info.csv")
test_df = pd.read_csv("/kaggle/input/datastorm-competition/Testing-data.csv")

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
historical_transaction_data = pd.read_csv("Datasets\Historical-transaction-data.csv")
store_info_data = pd.read_csv("Datasets\Store-info.csv")
test_df = pd.read_csv("Datasets\Testing-data.csv")

In [11]:
historical_transaction_data.head()

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold
0,ORANGE BARLEY 1.5L,2021-12-11T00:00:00.000Z,147.0,BGXA,SHOP008,220,2
1,GINGER BEER 1.5L,2021-10-17T00:00:00.000Z,371.0,IA25,SHOP112,220,2
2,TONIC PET 500ML,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,160,2
3,CREAM SODA 1L,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,150,2
4,STRAWBERRY MILK 180ML,2021-10-23T00:00:00.000Z,1310.0,7S00,SHOP112,210,5


In [12]:
historical_transaction_data.dtypes

item_description     object
transaction_date     object
invoice_id          float64
customer_id          object
shop_id              object
item_price            int64
quantity_sold         int64
dtype: object

In [13]:
historical_transaction_data["shop_id"].nunique()

124

# Working with item_description feature

In [14]:
historical_transaction_data["item_description"].unique()

array(['ORANGE BARLEY 1.5L', 'GINGER BEER 1.5L', 'TONIC PET 500ML',
       'CREAM SODA 1L', 'STRAWBERRY MILK 180ML', 'CREAM SODA 1.5L',
       'TWISTEE APPLE 1L', 'FIT O MIXED FRUIT 200ML', 'SODA PET 1.5L',
       'FIT O MIXED FRUIT 1L', 'KIK COLA 1L', 'LEMONADE 500ML',
       'FIT O ORANGE 1L', 'NECTO 1.5L', 'LEMONADE 1.5L', 'GINGER BEER 1L',
       'FIT O MANGO 200ML', 'FIT O MANGO 1L', 'CREAM SODA 500ML',
       'FIT O ORANGE 200ML', nan, 'CREAM SODA APPLE POP 1L',
       'BOTTLED DRINKING WATER 500ML', 'SODA 500ML', 'GINGER BEER 500ML',
       'SODA 1L', 'NECTO 500ML', 'ORANGE CRUSH 500ML',
       'BOTTLED DRINKING WATER 1.5L', 'NECTO 1L',
       'GINGER BEER SUGAR FREE 500ML', 'ORANGE CRUSH 1.5L',
       'DRY GINGER ALE PET 500ML', 'LEMONADE 1L', 'ORANGE CRUSH 1L',
       'GINGER BEER 400ML', 'CHOCOLATE MILK 180ML', 'LIME CRUSH JUICE 1L'],
      dtype=object)

In [15]:
historical_transaction_data["item_description"].nunique()

37

In [16]:
train_df = historical_transaction_data.copy(deep=True)

In [17]:
train_df.head()

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold
0,ORANGE BARLEY 1.5L,2021-12-11T00:00:00.000Z,147.0,BGXA,SHOP008,220,2
1,GINGER BEER 1.5L,2021-10-17T00:00:00.000Z,371.0,IA25,SHOP112,220,2
2,TONIC PET 500ML,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,160,2
3,CREAM SODA 1L,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,150,2
4,STRAWBERRY MILK 180ML,2021-10-23T00:00:00.000Z,1310.0,7S00,SHOP112,210,5


In [18]:
train_df[['item_name', 'item_quantity']] = train_df['item_description'].str.rsplit(' ', n=1, expand=True)

In [19]:
train_df.head()

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold,item_name,item_quantity
0,ORANGE BARLEY 1.5L,2021-12-11T00:00:00.000Z,147.0,BGXA,SHOP008,220,2,ORANGE BARLEY,1.5L
1,GINGER BEER 1.5L,2021-10-17T00:00:00.000Z,371.0,IA25,SHOP112,220,2,GINGER BEER,1.5L
2,TONIC PET 500ML,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,160,2,TONIC PET,500ML
3,CREAM SODA 1L,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,150,2,CREAM SODA,1L
4,STRAWBERRY MILK 180ML,2021-10-23T00:00:00.000Z,1310.0,7S00,SHOP112,210,5,STRAWBERRY MILK,180ML


### Covert the values in the item_quantity to liters

In [20]:
def convert_to_liters(value):
    try:
        if value[-2:] == "ML":
            return float(value[:-2])/1000
        else:
            return float(value[:-1])
    except:
        return np.NaN

In [21]:
train_df['item_quantity_in_liters'] = train_df['item_quantity'].apply(lambda x: convert_to_liters(x))

In [22]:
train_df['item_quantity_in_liters'].head()

0    1.50
1    1.50
2    0.50
3    1.00
4    0.18
Name: item_quantity_in_liters, dtype: float64

### Drop the item_quantity column

In [23]:
train_df.drop(['item_quantity'], axis=1, inplace=True)


In [24]:
train_df.head()

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold,item_name,item_quantity_in_liters
0,ORANGE BARLEY 1.5L,2021-12-11T00:00:00.000Z,147.0,BGXA,SHOP008,220,2,ORANGE BARLEY,1.50
1,GINGER BEER 1.5L,2021-10-17T00:00:00.000Z,371.0,IA25,SHOP112,220,2,GINGER BEER,1.50
2,TONIC PET 500ML,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,160,2,TONIC PET,0.50
3,CREAM SODA 1L,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,150,2,CREAM SODA,1.00
4,STRAWBERRY MILK 180ML,2021-10-23T00:00:00.000Z,1310.0,7S00,SHOP112,210,5,STRAWBERRY MILK,0.18


### Drop the item_description column

In [25]:
train_df.drop(['item_description'], axis=1, inplace=True)

In [26]:
train_df.head()

,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold,item_name,item_quantity_in_liters
0,2021-12-11T00:00:00.000Z,147.0,BGXA,SHOP008,220,2,ORANGE BARLEY,1.50
1,2021-10-17T00:00:00.000Z,371.0,IA25,SHOP112,220,2,GINGER BEER,1.50
2,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,160,2,TONIC PET,0.50
3,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,150,2,CREAM SODA,1.00
4,2021-10-23T00:00:00.000Z,1310.0,7S00,SHOP112,210,5,STRAWBERRY MILK,0.18


# Working on customers interest

In [27]:
train_df.head()

,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold,item_name,item_quantity_in_liters
0,2021-12-11T00:00:00.000Z,147.0,BGXA,SHOP008,220,2,ORANGE BARLEY,1.50
1,2021-10-17T00:00:00.000Z,371.0,IA25,SHOP112,220,2,GINGER BEER,1.50
2,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,160,2,TONIC PET,0.50
3,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,150,2,CREAM SODA,1.00
4,2021-10-23T00:00:00.000Z,1310.0,7S00,SHOP112,210,5,STRAWBERRY MILK,0.18


In [28]:
train_df['customer_id'].nunique()

191636

In [29]:
unique_customers = train_df[["shop_id", "customer_id"]].groupby("shop_id")["customer_id"].nunique()

In [30]:
unique_customers_df = unique_customers.to_frame().reset_index()

In [31]:
unique_customers_df.head()

,shop_id,customer_id
0,SHOP001,1118
1,SHOP002,2577
2,SHOP003,2064
3,SHOP004,1744
4,SHOP005,2203


In [32]:
unique_customers_df = unique_customers_df.rename(columns={'customer_id': 'unique_customers'})

In [33]:
unique_customers_df.head()

,shop_id,unique_customers
0,SHOP001,1118
1,SHOP002,2577
2,SHOP003,2064
3,SHOP004,1744
4,SHOP005,2203


In [34]:
customer_info_df = pd.merge(unique_customers_df, store_info_data, on='shop_id')

In [35]:
customer_info_df.head()

,shop_id,unique_customers,shop_area_sq_ft,shop_profile
0,SHOP001,1118,580,Low
1,SHOP002,2577,637,NaN
2,SHOP003,2064,810,High
3,SHOP004,1744,516,Low
4,SHOP005,2203,617,Low


In [36]:
store_info_data = customer_info_df.copy(deep=True)

In [37]:
store_info_data.head()

,shop_id,unique_customers,shop_area_sq_ft,shop_profile
0,SHOP001,1118,580,Low
1,SHOP002,2577,637,NaN
2,SHOP003,2064,810,High
3,SHOP004,1744,516,Low
4,SHOP005,2203,617,Low


# Fix Quantity

In [38]:
min_prices = train_df[["item_name", "item_price"]].groupby("item_name")["item_price"].min().to_dict()

# if item_name is in min_prices, then use min_prices[item_name] to calculate the quantity sold or use the existing quantity sold

def calculate_quantity_sold(row):
    if row['item_name'] in min_prices:
        return row['item_price']//min_prices[row['item_name']]
    else:
        return row['quantity_sold']
    
train_df['quantity_sold'] = train_df.apply(lambda row: calculate_quantity_sold(row), axis=1)

In [39]:
train_df["customer_id"].nunique()

191636

# Working with Sales

In [40]:
train_df.head()

,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold,item_name,item_quantity_in_liters
0,2021-12-11T00:00:00.000Z,147.0,BGXA,SHOP008,220,1,ORANGE BARLEY,1.50
1,2021-10-17T00:00:00.000Z,371.0,IA25,SHOP112,220,2,GINGER BEER,1.50
2,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,160,1,TONIC PET,0.50
3,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,150,1,CREAM SODA,1.00
4,2021-10-23T00:00:00.000Z,1310.0,7S00,SHOP112,210,3,STRAWBERRY MILK,0.18


In [41]:
tot_sales = train_df[["shop_id", "item_price"]].groupby("shop_id")["item_price"].sum()

In [42]:
tot_sales_df = tot_sales.to_frame().reset_index()
tot_sales_df.head()

,shop_id,item_price
0,SHOP001,433145
1,SHOP002,1090135
2,SHOP003,769685
3,SHOP004,711050
4,SHOP005,842995


In [43]:
store_info_data.head()

,shop_id,unique_customers,shop_area_sq_ft,shop_profile
0,SHOP001,1118,580,Low
1,SHOP002,2577,637,NaN
2,SHOP003,2064,810,High
3,SHOP004,1744,516,Low
4,SHOP005,2203,617,Low


In [44]:
tot_sales_df = tot_sales_df.rename(columns={'item_price': 'total_sales'})

In [45]:
tot_sales_df.head()

,shop_id,total_sales
0,SHOP001,433145
1,SHOP002,1090135
2,SHOP003,769685
3,SHOP004,711050
4,SHOP005,842995


In [46]:
store_info_df = pd.merge(tot_sales_df, store_info_data, on='shop_id')

In [47]:
store_info_df.head()

,shop_id,total_sales,unique_customers,shop_area_sq_ft,shop_profile
0,SHOP001,433145,1118,580,Low
1,SHOP002,1090135,2577,637,NaN
2,SHOP003,769685,2064,810,High
3,SHOP004,711050,1744,516,Low
4,SHOP005,842995,2203,617,Low


In [48]:
store_info_data = store_info_df.copy(deep=True)

In [49]:
store_info_data.head()

,shop_id,total_sales,unique_customers,shop_area_sq_ft,shop_profile
0,SHOP001,433145,1118,580,Low
1,SHOP002,1090135,2577,637,NaN
2,SHOP003,769685,2064,810,High
3,SHOP004,711050,1744,516,Low
4,SHOP005,842995,2203,617,Low


# Work on date

In [50]:
# Seperate date into day, month, year, time and day of week
train_df['transaction_date'] = pd.to_datetime(train_df['transaction_date'])
train_df['transaction_day'] = train_df['transaction_date'].dt.day
train_df['transaction_month'] = train_df['transaction_date'].dt.month
train_df['transaction_year'] = train_df['transaction_date'].dt.year
train_df['transaction_time'] = train_df['transaction_date'].dt.time
train_df['transaction_day_of_week'] = train_df['transaction_date'].dt.dayofweek

train_df.head()

,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold,item_name,item_quantity_in_liters,transaction_day,transaction_month,transaction_year,transaction_time,transaction_day_of_week
0,2021-12-11 00:00:00+00:00,147.0,BGXA,SHOP008,220,1,ORANGE BARLEY,1.50,11,12,2021,00:00:00,5
1,2021-10-17 00:00:00+00:00,371.0,IA25,SHOP112,220,2,GINGER BEER,1.50,17,10,2021,00:00:00,6
2,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,160,1,TONIC PET,0.50,13,12,2021,00:00:00,0
3,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,150,1,CREAM SODA,1.00,13,12,2021,00:00:00,0
4,2021-10-23 00:00:00+00:00,1310.0,7S00,SHOP112,210,3,STRAWBERRY MILK,0.18,23,10,2021,00:00:00,5


In [51]:
# Print the number of unique values in each column
for col in train_df.columns:
    print(col, train_df[col].nunique())
    

transaction_date 62
invoice_id 343342
customer_id 191636
shop_id 124
item_price 209
quantity_sold 71
item_name 21
item_quantity_in_liters 6
transaction_day 31
transaction_month 3
transaction_year 1
transaction_time 1
transaction_day_of_week 7


In [52]:
# Remove time and year
train_df.drop(['transaction_time', 'transaction_year'], axis=1, inplace=True)

In [53]:
train_df.head()

,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold,item_name,item_quantity_in_liters,transaction_day,transaction_month,transaction_day_of_week
0,2021-12-11 00:00:00+00:00,147.0,BGXA,SHOP008,220,1,ORANGE BARLEY,1.50,11,12,5
1,2021-10-17 00:00:00+00:00,371.0,IA25,SHOP112,220,2,GINGER BEER,1.50,17,10,6
2,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,160,1,TONIC PET,0.50,13,12,0
3,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,150,1,CREAM SODA,1.00,13,12,0
4,2021-10-23 00:00:00+00:00,1310.0,7S00,SHOP112,210,3,STRAWBERRY MILK,0.18,23,10,5


# Work on most popular beverage

In [54]:
# Get most popular item per shop

item_counts = train_df.groupby(["shop_id", "item_name"])["quantity_sold"].sum()

most_popular_items = item_counts.groupby("shop_id").idxmax().reset_index()
most_popular_items.columns = ["shop_id", "most_sold_item"]
most_popular_items["most_sold_item"] = most_popular_items["most_sold_item"].str[1]

most_popular_items.count()

shop_id           124
most_sold_item    124
dtype: int64

In [55]:
# Merge with store info
store_info_data = pd.merge(store_info_data, most_popular_items, on="shop_id")

store_info_data.head()

,shop_id,total_sales,unique_customers,shop_area_sq_ft,shop_profile,most_sold_item
0,SHOP001,433145,1118,580,Low,GINGER BEER
1,SHOP002,1090135,2577,637,NaN,CREAM SODA
2,SHOP003,769685,2064,810,High,GINGER BEER
3,SHOP004,711050,1744,516,Low,GINGER BEER
4,SHOP005,842995,2203,617,Low,GINGER BEER


# Average time between orders

In [56]:
train_df_ = train_df.copy(deep=True)

train_df_.sort_values(['customer_id','transaction_date'], inplace=True)

train_df_

,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold,item_name,item_quantity_in_liters,transaction_day,transaction_month,transaction_day_of_week
452281,2021-10-16 00:00:00+00:00,370560.0,0.00E+00,SHOP010,440,2,ORANGE BARLEY,1.5,16,10,5
308101,2021-10-26 00:00:00+00:00,61463.0,0.00E+00,SHOP034,880,4,ORANGE BARLEY,1.5,26,10,1
454752,2021-10-27 00:00:00+00:00,1328902.0,0.00E+00,SHOP111,270,4,FIT O ORANGE,1.0,27,10,2
333814,2021-10-29 00:00:00+00:00,668008.0,0.00E+00,SHOP081,110,1,SODA,1.0,29,10,4
333815,2021-10-29 00:00:00+00:00,668008.0,0.00E+00,SHOP081,150,1,GINGER BEER,1.0,29,10,4
...,...,...,...,...,...,...,...,...,...,...,...
202042,2021-11-21 00:00:00+00:00,213333.0,ZZZW,SHOP073,160,1,SODA PET,1.5,21,11,6
202043,2021-11-21 00:00:00+00:00,213333.0,ZZZW,SHOP073,220,2,GINGER BEER,1.5,21,11,6
234007,2021-11-25 00:00:00+00:00,3233435.0,ZZZW,SHOP074,220,2,GINGER BEER,1.5,25,11,3
463571,2021-12-04 00:00:00+00:00,294745.0,ZZZW,SHOP074,160,1,SODA PET,1.5,4,12,5


In [57]:
train_df_['time_since_last_purchase']  = train_df_.groupby('customer_id')['transaction_date'].diff().dt.days
train_df_

,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold,item_name,item_quantity_in_liters,transaction_day,transaction_month,transaction_day_of_week,time_since_last_purchase
452281,2021-10-16 00:00:00+00:00,370560.0,0.00E+00,SHOP010,440,2,ORANGE BARLEY,1.5,16,10,5,NaN
308101,2021-10-26 00:00:00+00:00,61463.0,0.00E+00,SHOP034,880,4,ORANGE BARLEY,1.5,26,10,1,10.0
454752,2021-10-27 00:00:00+00:00,1328902.0,0.00E+00,SHOP111,270,4,FIT O ORANGE,1.0,27,10,2,1.0
333814,2021-10-29 00:00:00+00:00,668008.0,0.00E+00,SHOP081,110,1,SODA,1.0,29,10,4,2.0
333815,2021-10-29 00:00:00+00:00,668008.0,0.00E+00,SHOP081,150,1,GINGER BEER,1.0,29,10,4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
202042,2021-11-21 00:00:00+00:00,213333.0,ZZZW,SHOP073,160,1,SODA PET,1.5,21,11,6,NaN
202043,2021-11-21 00:00:00+00:00,213333.0,ZZZW,SHOP073,220,2,GINGER BEER,1.5,21,11,6,0.0
234007,2021-11-25 00:00:00+00:00,3233435.0,ZZZW,SHOP074,220,2,GINGER BEER,1.5,25,11,3,4.0
463571,2021-12-04 00:00:00+00:00,294745.0,ZZZW,SHOP074,160,1,SODA PET,1.5,4,12,5,9.0


In [58]:
avg_time_between_purchases = train_df_.groupby('shop_id')['time_since_last_purchase'].mean().reset_index()

In [59]:
# Merge with store info
store_info_data = pd.merge(store_info_data, avg_time_between_purchases, on="shop_id")
store_info_data.head()

,shop_id,total_sales,unique_customers,shop_area_sq_ft,shop_profile,most_sold_item,time_since_last_purchase
0,SHOP001,433145,1118,580,Low,GINGER BEER,8.704721
1,SHOP002,1090135,2577,637,NaN,CREAM SODA,7.877802
2,SHOP003,769685,2064,810,High,GINGER BEER,8.612475
3,SHOP004,711050,1744,516,Low,GINGER BEER,8.586707
4,SHOP005,842995,2203,617,Low,GINGER BEER,8.415926


# Working with number of customers that came more than once

In [60]:
# Get the customers that came more than once to the corresponding shop
# number of customers that came more than once
customers_that_came_more_than_once = train_df_.groupby('shop_id')['customer_id'].apply(lambda x: x[x.duplicated()].nunique()).reset_index()
# customers_that_came_more_than_once = train_df_.groupby('shop_id')['customer_id'].apply(lambda x: x[x.duplicated()].unique()).reset_index()

In [61]:
customers_that_came_more_than_once.head()

,shop_id,customer_id
0,SHOP001,402
1,SHOP002,930
2,SHOP003,706
3,SHOP004,689
4,SHOP005,822


In [62]:
customers_that_came_more_than_once = customers_that_came_more_than_once.rename(
    columns={"customer_id": "came_more_than_once"}
)

In [63]:
customers_that_came_more_than_once.head()

,shop_id,came_more_than_once
0,SHOP001,402
1,SHOP002,930
2,SHOP003,706
3,SHOP004,689
4,SHOP005,822


In [64]:
customer_visit_df = pd.merge(customers_that_came_more_than_once, store_info_data, on='shop_id')

In [65]:
customer_visit_df.shape

(124, 8)

In [66]:
customer_visit_df.head()

,shop_id,came_more_than_once,total_sales,unique_customers,shop_area_sq_ft,shop_profile,most_sold_item,time_since_last_purchase
0,SHOP001,402,433145,1118,580,Low,GINGER BEER,8.704721
1,SHOP002,930,1090135,2577,637,NaN,CREAM SODA,7.877802
2,SHOP003,706,769685,2064,810,High,GINGER BEER,8.612475
3,SHOP004,689,711050,1744,516,Low,GINGER BEER,8.586707
4,SHOP005,822,842995,2203,617,Low,GINGER BEER,8.415926


In [67]:
store_info_data = customer_visit_df.copy(deep=True)

In [68]:
train_df_2 = train_df_.copy(deep=True)

# Weekend boolean column
train_df_2['weekend'] = train_df_2['transaction_day_of_week'].apply(lambda x: 1 if x >= 4 else 0)

weekend_sales = train_df_2[train_df_2['weekend'] == 1].groupby('shop_id')['item_price'].sum().reset_index()
weekend_sales = weekend_sales.rename(columns={'item_price': 'weekend_sales'})

weekday_sales = train_df_2[train_df_2['weekend'] == 0].groupby('shop_id')['item_price'].sum().reset_index()
weekend_sales = weekend_sales.rename(columns={'item_price': 'weekday_sales'})


store_info_data = pd.merge(store_info_data, weekend_sales, on="shop_id")
store_info_data = pd.merge(store_info_data, weekday_sales, on="shop_id")

store_info_data.head()

,shop_id,came_more_than_once,total_sales,unique_customers,shop_area_sq_ft,shop_profile,most_sold_item,time_since_last_purchase,weekend_sales,item_price
0,SHOP001,402,433145,1118,580,Low,GINGER BEER,8.704721,212520,220625
1,SHOP002,930,1090135,2577,637,NaN,CREAM SODA,7.877802,521480,568655
2,SHOP003,706,769685,2064,810,High,GINGER BEER,8.612475,394235,375450
3,SHOP004,689,711050,1744,516,Low,GINGER BEER,8.586707,386490,324560
4,SHOP005,822,842995,2203,617,Low,GINGER BEER,8.415926,445910,397085
